<a href="https://colab.research.google.com/github/Divyam-goel17/Personality_recognition_model/blob/main/personality_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install catboost

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.metrics import classification_report, f1_score
import pickle
import os.path
import plotly.offline as pyo
import plotly.graph_objs as go
import spacy
from nltk.stem.snowball import SnowballStemmer
from catboost import CatBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

Loading and visualizing the dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/mbti_1.csv')

In [ ]:
df.head(10)

,posts,type
0,know intj tool use interaction people excuse a...,INTJ
1,rap music ehh opp yeah know valid well know fa...,INTJ
2,preferably p hd low except wew lad video p min...,INTJ
3,drink like wish could drink red wine give head...,INTJ
4,space program ah bad deal meing freelance max ...,INTJ
5,loose stop overthinking everything ruin friend...,INTJ
6,often men one put chance get burn otherwise no...,INTJ
7,early definitely relate quiet self destruction...,INTJ
8,one full list would impossibly long quotable f...,INTJ
9,technique information power share unless absol...,INTJ


In [ ]:
df['posts'][0]
df['type'][0]

'INTJ'

In [ ]:
df['type'].unique()

array(['INTJ', 'INTP', 'ISFJ', 'ISFP', 'ISTJ', 'ISTP', 'ENFJ', 'ENFP',
       'ENTJ', 'ENTP', 'ESFJ', 'ESFP', 'ESTJ', 'ESTP', 'INFJ', 'INFP'],
      dtype=object)

In [ ]:
TYPES=['ENFJ','ENFP','ENTJ','ENTP','ESFJ','ESFP','ESTJ','ESTP','INFJ','INFP','INTJ','INTP','ISFJ','ISFP','ISTJ','ISTP']

In [ ]:
print(f"Total of {len(df['type'].unique())} types of classified MBTI posts")

Total of 16 types of classified MBTI posts


In [ ]:
df.isnull().sum()

,0
posts,0
type,0


In [ ]:
df_bar_chart=df.groupby('type').count()

In [ ]:
trace1 = go.Bar(x=df_bar_chart.index, y=df_bar_chart['posts'])

In [ ]:
data = [trace1]

In [ ]:
layout = go.Layout(title='MBTI # Classified Posts per Type')

In [ ]:
fig = go.Figure(data=data, layout=layout)

In [ ]:
fig.show()

In [ ]:
recreate_model=True

In [ ]:
filename = '/content/drive/MyDrive/Colab Notebooks/mbti_svm_v1.sav'

In [ ]:
X = df['posts'] # features
y = df['type'] # labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Check if need to recreate the model
if recreate_model:

  # Creating an instance to vectorizer:
  vectorizer = TfidfVectorizer()

  # Training the vectorizer:
  X_train_tfidf = vectorizer.fit_transform(X_train)

  # Training the classifier:
  clf = LinearSVC()
  clf.fit(X_train_tfidf, y_train)

  # Pipelining the vectorizer and the classifier
  text_clf = Pipeline([('tfidf',TfidfVectorizer()),('clf',LinearSVC())])
  text_clf.fit(X_train, y_train)

  # saving the model to disk
  pickle.dump(text_clf, open(filename, 'wb'))

  # If there is no need to recreate the model, just open the file from the disk else:
    # loading the model from disk
  text_clf = pickle.load(open(filename, 'rb'))


predictions = text_clf.predict(X_test)



Analysis of SVC Linear Model

In [ ]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

        ENFJ       0.84      0.58      0.69       319
        ENFP       0.82      0.78      0.80      1249
        ENTJ       0.90      0.80      0.84       577
        ENTP       0.86      0.83      0.84      2324
        ESFJ       0.83      0.45      0.59        33
        ESFP       0.88      0.48      0.62        75
        ESTJ       0.90      0.84      0.87       105
        ESTP       0.95      0.90      0.92       398
        INFJ       0.81      0.84      0.83      2954
        INFP       0.80      0.82      0.81      2391
        INTJ       0.83      0.87      0.85      4531
        INTP       0.84      0.87      0.86      5033
        ISFJ       0.80      0.61      0.69       132
        ISFP       0.81      0.60      0.69       161
        ISTJ       0.86      0.68      0.76       253
        ISTP       0.89      0.79      0.84       679

    accuracy                           0.84     21214
   macro avg       0.85   

In [ ]:
print(f"Overall accuracy of the model: {round(metrics.accuracy_score(y_test, predictions),2)}")

Overall accuracy of the model: 0.84


In [ ]:
str(text_clf.predict(["Hello world"])[0])

'INFP'

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
class Lemmatizer(object):
  def __init__(self):
    self.lemmatizer = WordNetLemmatizer()
  def __call__(self, sentence):
    return [self.lemmatizer.lemmatize(word) for word in sentence.split() if len(word)>2]


In [ ]:
import pickle
target_encoder=LabelEncoder()
vectorizer=TfidfVectorizer( max_features=5000,stop_words='english',tokenizer=Lemmatizer())
vectorizer.fit(X_train)
X_test=vectorizer.transform(X_test).toarray()
y_test=target_encoder.fit_transform(y_test)

/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning:

The parameter 'token_pattern' will not be used since 'tokenizer' is not None'

/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:402: UserWarning:

Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'wa'] not in stop_words.



In [ ]:
filename2 = '/content/drive/MyDrive/Colab Notebooks/mbti_svm_v2.sav'
model_cat = pickle.load(open(filename2, 'rb'))

Analysis of CatBoost Model

In [ ]:
print('test classification report \n ',classification_report(y_test,model_cat.predict(X_test),target_names=target_encoder.inverse_transform([i for i in range(16)])))

test classification report 
                precision    recall  f1-score   support

        ENFJ       0.45      0.15      0.23       319
        ENFP       0.48      0.29      0.36      1249
        ENTJ       0.33      0.42      0.37       577
        ENTP       0.42      0.45      0.43      2324
        ESFJ       0.00      0.00      0.00        33
        ESFP       0.33      0.01      0.03        75
        ESTJ       0.05      0.10      0.07       105
        ESTP       0.20      0.52      0.29       398
        INFJ       0.14      0.08      0.10      2954
        INFP       0.13      0.12      0.12      2391
        INTJ       0.27      0.54      0.36      4531
        INTP       0.29      0.15      0.20      5033
        ISFJ       0.00      0.00      0.00       132
        ISFP       0.00      0.00      0.00       161
        ISTJ       0.03      0.01      0.02       253
        ISTP       0.06      0.03      0.04       679

    accuracy                           0.27     21

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [ ]:
print(accuracy_score(y_test,model_cat.predict(X_test)))

0.26751202036391064


In [ ]:
text=["Hello world"]
txt=vectorizer.transform(text).toarray()
ans=model_cat.predict(txt)
print(ans)
str(TYPES[ans[0][0]])

[[10]]


'INTJ'